# First-order logic (NLTK 10.3)

## 1. Warming up: representing first order logic

Let's first read some expressions in first-order logic. An expression is composed of arguments and a function.

In [1]:
import nltk
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument

<ConstantExpression angus>

We can also check their function which will become handy later when we discuss lambda calculus.

In [2]:
expr.argument.type

e

In [3]:
expr.function

<ConstantExpression walk>

In [4]:
expr.function.type

<e,?>

The the type checker tries to infer the type as much as it can. But we can tell it the type...

In [5]:
sig = {'walk': '<e, t>'} # a dictionary
expr = read_expr('walk(angus)', signature=sig)
expr.function.type

e

Hm, this should give us t. A bug?

First-order logic expressions may also contain variables. Let's parse some of those and check if they are bound or free. What is the difference?

In [6]:
read_expr('dog(cyril)').free()

set()

In [7]:
read_expr('dog(x)').free()

{Variable('x')}

In [8]:
read_expr('own(angus, cyril)').free()

set()

In [9]:
read_expr('exists x.dog(x)').free()

set()

In [10]:
read_expr('((some x. walk(x)) -> sing(x))').free()

{Variable('x')}

In [11]:
read_expr('exists x.own(y, x)').free()

{Variable('y')}

In [12]:
read_expr('all x.((exists x. dog(x)) -> bark(x))').free()

set()

In [13]:
read_expr('((exists x. dog(x)) -> bark(x))').free()

{Variable('x')}

The scope of the logical constants is controlled with brackets!

## 2. Evaluating first-order logic in a model

In [14]:
# Individuals in our model
dom = set(['b', 'o', 'c'])

# The assignment function or valuation
v = """ 
bertie => b 
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""

# Parse the evaluation and print it
val = nltk.Valuation.fromstring(v)
print(val) # A dictionary

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('o', 'c'), ('c', 'b')},
 'walk': {('c',), ('o',)}}


Let's see if a particular tuple is in valuation of a particular predicate. Does Olive see Cyril? Is Bertie a boy?

In [15]:
('o', 'c') in val['see']

True

In [16]:
('b',) in val['boy']

True

Variables must be assigned to objects in the domain before expressions can be evaluated. We can do this as follows. (In this case we would say we have a partial variable assignment or g function.)

In [17]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')]) # Partial variable assignment, this is optional
print(g)

g[c/y][o/x]


Let's avaluate some expressions with this assignement g.

In [18]:
m = nltk.Model(dom, val)
t = m.evaluate('see(olive, y)', g)
t1 = m.evaluate('see(y, x)', g)
print(t, t1)

True False


Check the assignment

In [19]:
print(g['y'])

c


Clear the assignment of variables. Now, the expression cannot be evaluated as there is no assignment.

In [20]:
g.purge()
print(g)
t = m.evaluate('walk(x)', g)
print(t)

g
Undefined


However, when the variable is quantified, the code starts with an empty assignment and assigns constants to variables.

In [21]:
print(m.evaluate('exists x.(girl(x) & walk(x))', g))

True


We can manually change the assignment

In [22]:
print(m.evaluate('girl(x) & walk(x)', g.add('x', 'o')))

True


We might also ask what objects satisfy a formula.

In [23]:
phi = read_expr('girl(x) & walk(x)')
print(m.satisfiers(phi, 'x', g)) 

{'o'}


In [24]:
phi = read_expr('girl(x) | walk(x)')
print(m.satisfiers(phi, 'x', g)) 

{'o', 'c'}


Natural language pressupositions and logic

In [25]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

{'b', 'c', 'o'}

Note that the expression is equivalent to -girl(x) | walk(x). This means that it is also satisifed by someone who is not a girl (and does or does not walk): b and c. The expression is satisfied by every member in the domain but there is only one girl.

In [26]:
print(m.evaluate('all x.(girl(x) -> walk(x))', g))

True


In [27]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}